## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-11-04-01-24 +0000,nypost,Protester fires gun at federal agents in chaot...,https://nypost.com/2025/07/11/us-news/californ...
1,2025-07-11-03-35-21 +0000,nypost,Grieving mom of dead Camp Mystic camper desper...,https://nypost.com/2025/07/10/us-news/grieving...
2,2025-07-11-03-34-23 +0000,bbc,War crimes are likely being committed in Darfu...,https://www.bbc.com/news/articles/cvg819yxkp5o
3,2025-07-11-02-54-19 +0000,nypost,Physical therapist burned alive inside hyperba...,https://nypost.com/2025/07/10/us-news/physical...
4,2025-07-11-02-39-47 +0000,bbc,Watch: Smoke billows from luxury yacht fire in...,https://www.bbc.com/news/videos/c4g85005q93o


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2286/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
55,trump,18
69,tariffs,6
201,nyc,4
58,where,4
322,brazil,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
46,2025-07-10-16-08-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,46
53,2025-07-10-12-04-11 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...,38
5,2025-07-11-02-31-43 +0000,nypost,Trump admin still deciding where it would depo...,https://nypost.com/2025/07/10/us-news/trump-ad...,36
7,2025-07-11-02-08-37 +0000,nypost,Trump announces 35% tariffs on imports from Ca...,https://nypost.com/2025/07/10/us-news/trump-an...,33
54,2025-07-10-11-30-07 +0000,bbc,Brazil vows to match US tariffs after Trump th...,https://www.bbc.com/news/articles/c62dvyv60z7o,33


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
46,46,2025-07-10-16-08-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
1,24,2025-07-11-03-35-21 +0000,nypost,Grieving mom of dead Camp Mystic camper desper...,https://nypost.com/2025/07/10/us-news/grieving...
0,21,2025-07-11-04-01-24 +0000,nypost,Protester fires gun at federal agents in chaot...,https://nypost.com/2025/07/11/us-news/californ...
19,21,2025-07-10-23-49-13 +0000,nypost,Hakeem Jeffries’ team blasts Zohran Mamdani’s ...,https://nypost.com/2025/07/10/us-news/hakeem-j...
17,19,2025-07-11-00-03-19 +0000,nypost,Mayor Adams must implement controversial and c...,https://nypost.com/2025/07/10/us-news/mayor-ad...
13,16,2025-07-11-00-45-44 +0000,nypost,Search-and-rescue volunteers had to rely on se...,https://nypost.com/2025/07/10/us-news/search-a...
8,15,2025-07-11-01-57-20 +0000,nypost,Florida scammer posing as doctor gave Botox in...,https://nypost.com/2025/07/10/us-news/florida-...
5,14,2025-07-11-02-31-43 +0000,nypost,Trump admin still deciding where it would depo...,https://nypost.com/2025/07/10/us-news/trump-ad...
11,14,2025-07-11-00-52-06 +0000,nypost,Hochul blames Trump’s ‘Big Beautiful’ bill for...,https://nypost.com/2025/07/10/us-news/hochul-b...
26,14,2025-07-10-22-54-22 +0000,nypost,Long Island’s alleged ghost gun kingpin arrest...,https://nypost.com/2025/07/10/us-news/long-isl...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
